 "# Green Leaf Segmentation and NDVI Extraction\n",
    "This notebook segments green leaves from hyperspectral ENVI images, extracts average spectra, saves masks and NDVI images, and exports results to Excel."

In [ ]:
import os
import numpy as np
import spectral
import pandas as pd
import matplotlib.pyplot as plt
import scipy.ndimage

User settings

In [ ]:
folder = r"F:/NPEC/20250717/1/SNAPSCAN/162426471044/Data"

Segmentation and NDVI functions

In [ ]:
# Define green segmentation function (simple NDVI or band thresholding)
def segment_green_leaves(data, camera_type):
    # plastic mask: use band 97 to seperate plant from plastic
    plastic  = np.sum(data[:, :, 97].astype(np.float32), axis=2)
    plastic_mask = plastic > 0.2

    # NDVI mask: Red = sum of bands 58-62, NIR = sum of bands 103-107
    red = np.sum(data[:, :, 58:63].astype(np.float32), axis=2)
    nir = np.sum(data[:, :, 103:108].astype(np.float32), axis=2)
    ndvi = (nir - red) / (nir + red + 1e-8)
    ndvi_mask = ndvi > 0.4

    # Combine both masks (logical AND)
    mask = np.logical_and(plastic_mask, ndvi_mask)


    # Remove small objects (connected areas < 100 pixels)
    labeled_mask, num_features = scipy.ndimage.label(mask)
    sizes = np.bincount(labeled_mask.ravel())
    mask_clean = mask.copy()
    # sizes[0] is background, skip it
    for i, size in enumerate(sizes):
        if i == 0:
            continue
        if size < 100:
            mask_clean[labeled_mask == i] = 0

    return mask_clean

def NDVI_image(data):
    # NDVI mask: Red = sum of bands 58-62, NIR = sum of bands 103-107
    red = np.sum(data[:, :, 58:63].astype(np.float32), axis=2)
    nir = np.sum(data[:, :, 103:108].astype(np.float32), axis=2)
    ndvi = (nir - red) / (nir + red + 1e-8)
    # Restrict NDVI values to [0, 1]
    ndvi_restricted = np.clip(ndvi, 0, 1)
    return ndvi_restricted


Main procesing loop. Load in each image named XXX_refl.hdr/raw and produce an NDVI, mask, mean and average spectra 

In [ ]:
results = []

for file in os.listdir(folder):
    if file.endswith('refl.hdr'):
        
        # Load hyperspectral image
        base = os.path.splitext(file)[0]
        hdr_path = os.path.join(folder, file)
        raw_path = os.path.join(folder, base + '.raw')
        if not os.path.exists(raw_path):
            continue

        img = spectral.open_image(hdr_path)
        data = img.load()

        # Crop out the first 76 rows because there is no White reference for that part of the image.
        data = data[76:, :, :]

        mask = segment_green_leaves(data, camera_type)
        if np.sum(mask) == 0:
            continue  # Skip if no green pixels found

        # Ensure mask is 2D and matches the first two dimensions of data
        mask_2d = mask if mask.ndim == 2 else np.squeeze(mask)

        # Crop image to just the area that has masked pixels
        rows, cols = np.where(mask_2d)
        if len(rows) == 0 or len(cols) == 0:
            continue  # No green pixels found after mask
        min_row, max_row = rows.min(), rows.max()
        min_col, max_col = cols.min(), cols.max()
        cropped_data = data[min_row:max_row+1, min_col:max_col+1, :]
        cropped_mask = mask_2d[min_row:max_row+1, min_col:max_col+1]

        green_pixels = cropped_data[cropped_mask, :]  # Select all bands for masked pixels
        avg_spectra = np.mean(green_pixels, axis=0)
        std_spectra = np.std(green_pixels, axis=0)
        
        # Save mask for inspection
        mask_save_path = os.path.join(folder, f"{base}_mask.png")
        plt.imsave(mask_save_path, np.flipud(np.squeeze(cropped_mask.astype(np.uint8) * 255)), cmap='gray')


        # Save NDVI image for inspection
        ndvi_img = NDVI_image(cropped_data)
        ndvi_save_path = os.path.join(folder, f"{base}_ndvi.png")
        plt.imsave(ndvi_save_path, np.flipud(ndvi_img), cmap='cividis')

        results.append({
            "filename": file,
            "avg_spectra": avg_spectra,
            "std_spectra": std_spectra
        })

Save Results to Excel!

In [ ]:
# Prepare DataFrame for Excel
if results:
    wavelengths = img.metadata.get('wavelength', None)
    if isinstance(wavelengths, str):
        wavelengths = [w.strip() for w in wavelengths.strip('{}').split(',')]
    df = pd.DataFrame({
        "filename": [r["filename"] for r in results],
        **{f"avg_{w}": [r["avg_spectra"][i] for r in results] for i, w in enumerate(wavelengths)},
        **{f"std_{w}": [r["std_spectra"][i] for r in results] for i, w in enumerate(wavelengths)}
    })
    out_path = os.path.join(folder, f"{camera_type}_leaf_spectra.xlsx")
    df.to_excel(out_path, index=False)
    print(f"Saved results to {out_path}")
else:
    print("No green leaves found in any image.")